In [7]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from models.addmul import HandleAddMul

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

network_cache_dir = "networks/cache-networks/"
network_name = "lyr256-split0.8-lr0.01-add-mul.data"

checkpoint = True
test_flag = 1

input_dims = [42]
output_dims = [20]
batchsize = 128
num_epochs = 1
            

## generate mask on addition

In [8]:
'''Load Data'''
train_split = 0.8
test_split = 1 - train_split
data_fp = "generate_datasets/tmp/digit-data/simple_add.npy"
data = np.load(data_fp, allow_pickle=True)
data_len = len(data)
train_split_idx = int(data_len * train_split)
train_data = data[:train_split_idx]
test_data = data[train_split_idx:]

train_loader = torch.utils.data.DataLoader(dataset=torch.tensor(train_data), batch_size=batchsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=torch.Tensor(test_data), batch_size=batchsize, shuffle=True)

iterator_train = iter(cycle(train_loader))
iterator_test = iter(cycle(test_loader))


In [11]:
'''Initialise logits & define loss and Optimiser'''
handler = HandleAddMul(input_dims, output_dims, dir=network_cache_dir+network_name, 
                       checkpoint=checkpoint, use_optimiser=False)
handler.refreeze_weights()

logits = []
for layer in handler.network.layers[0]:
    if isinstance(layer, torch.nn.Linear):
        layer.weight.data = layer.weight.detach()
        inter = torch.full_like(layer.weight, 0.9)
        logits.append(torch.nn.Parameter(inter, requires_grad=True))

for l in logits:    
    l.requires_grad_(requires_grad=True)
        
criterion = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(logits, lr=0.01)

'''Initialise hyper-parameters'''
NUM_EPOCHS = 20000  # NB: check for number of training epochs in paper
tau = 1  # temperature parameter, NB: check for value in paper
alpha = 0.0001/128  # regularisation parameter, NB: check for value in paper
loss_trace = []
accuracy_trace = []
avg_loss_trace = []
avg_accuracy_trace = []

'''Mask Training'''
for e in range(NUM_EPOCHS):
    print(f'Starting epoch {e}...')
    
    '''Reload weights'''
    handler.network.load_save()
    handler.refreeze_weights()
    
    '''Sampling and generating masks.'''
    U1 = torch.rand(1, requires_grad=False).to(handler.network.device)
    U2 = torch.rand(1, requires_grad=False).to(handler.network.device)
    
    
    '''Gumbel Sigmoid & Straight through'''
    samples = []
    for layer in logits:
        samples.append(torch.sigmoid((layer - torch.log(torch.log(U1) / torch.log(U2))) / tau))
        
    binaries_stop = []
    for layer in samples:
        with torch.no_grad():
            binaries_stop.append((layer > 0.5).float() - layer)
        
    binaries = []
    iterator_samples = iter(samples)
          
    for idx, layer in enumerate(binaries_stop):
        binaries.append(layer + next(iterator_samples))
    terator_binary = iter(binaries)
    
    '''Inference with masked network and backpropagation.'''
    batch = next(iterator_train)
    # Load in batch data (not binaries for one-hot input)
    inp = torch.stack([torch.stack([b[0], b[1]]) for b in batch])
    otp = torch.stack([b[2] for b in batch]).to(handler.network.device)
    
    ops = torch.stack([b[3] for b in batch])
    # Convert batch data toone-hot representation
    inp, otp_ = handler.set_batched_digits(inp, otp, ops)
    inp_ = inp.to(handler.network.device)
    otp_ = otp_.to(handler.network.device)
    
    '''Pass batch data through masked net'''
    for layer in handler.network.layers[0]:
        if isinstance(layer, torch.nn.Linear):
            b = layer.bias
            w = layer.weight
            m = next(terator_binary)
            inp_ = handler.network.forward_mask_layer(inp_, m, w, b)
        else:
            inp_ = layer(inp_)
    otp_pred = inp_       
    assert otp_pred.is_leaf == False
    
    '''Fetch Running Accuracy'''
    with torch.no_grad():
        otp_stck = torch.stack([otp_pred[:,:10], otp_pred[:,10:]])
        otp_argmax = torch.argmax(otp_stck, dim=2)
        otp_class = otp_argmax[0]*10 + otp_argmax[1]
        diff = otp_class - otp
        cnt = len(diff[abs(diff) == 0])
        accuracy_trace.append(cnt/(len(diff)))
        
    '''Apply regularisation to CELoss'''
    all_logits = alpha*torch.cat([layer.view(-1) for layer in logits]).to(handler.network.device)
    optimiser.zero_grad()
    loss = criterion(otp_pred, otp_).to(handler.network.device) + torch.sum(all_logits).detach()
    loss.backward()
    optimiser.step()

    for layer in handler.network.layers[0]:
        if isinstance(layer, torch.nn.Linear):
            assert layer.weight.grad == None
    
            
    loss_trace.append(loss.item())
    avg_loss_trace.append(np.mean(loss_trace[-100:]))
    avg_accuracy_trace.append(np.mean(accuracy_trace[-10:]))
    if e % 100 == 0:        
        plt.cla()
        plt.clf()
        plt.plot(loss_trace)
        plt.plot(avg_loss_trace)
        plt.savefig('lossplot_add.png')
        plt.cla()
        plt.clf()
        plt.plot(accuracy_trace)
        plt.plot(avg_accuracy_trace)
        plt.savefig('accuracyplot_add.png')
        plt.cla()
        plt.clf()
        plt.close()
    if e % 100 == 0:
        print('Saving Mask...')
        torch.save(logits, 'masks/trained_logits_add_mask_.pt') 

... FNN Network training on cuda:0 ...
Accessing : networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
Starting epoch 0...
tensor([[0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        ...,
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
Saving Mask...
Starting epoch 1...
Starting epoch 2...
Starting epoch 3...
Starting epoch 4...
Starting epoch 5...
Starting epoch 6...
Starting epoch 7...
Starting epoch 8...
Starting epoch 9...
Starting epoch 10...
Starting epoch 11...
Starting epoch 12...
Starting epoch 13...
Starting epoch 14...
Starting epoch 15...
Starting epoch 16...
Starting epoch 17

Saving Mask...
Starting epoch 301...
Starting epoch 302...
Starting epoch 303...
Starting epoch 304...
Starting epoch 305...
Starting epoch 306...
Starting epoch 307...
Starting epoch 308...
Starting epoch 309...
Starting epoch 310...
Starting epoch 311...
Starting epoch 312...
Starting epoch 313...
Starting epoch 314...
Starting epoch 315...
Starting epoch 316...
Starting epoch 317...
Starting epoch 318...
Starting epoch 319...
Starting epoch 320...
Starting epoch 321...
Starting epoch 322...
Starting epoch 323...
Starting epoch 324...
Starting epoch 325...
Starting epoch 326...
Starting epoch 327...
Starting epoch 328...
Starting epoch 329...
Starting epoch 330...
Starting epoch 331...
Starting epoch 332...
Starting epoch 333...
Starting epoch 334...
Starting epoch 335...
Starting epoch 336...
Starting epoch 337...
Starting epoch 338...
Starting epoch 339...
Starting epoch 340...
Starting epoch 341...
Starting epoch 342...
Starting epoch 343...
Starting epoch 344...
Starting epoch 34

Starting epoch 610...
Starting epoch 611...
Starting epoch 612...
Starting epoch 613...
Starting epoch 614...
Starting epoch 615...
Starting epoch 616...
Starting epoch 617...
Starting epoch 618...
Starting epoch 619...
Starting epoch 620...
Starting epoch 621...
Starting epoch 622...
Starting epoch 623...
Starting epoch 624...
Starting epoch 625...
Starting epoch 626...
Starting epoch 627...
Starting epoch 628...
Starting epoch 629...
Starting epoch 630...
Starting epoch 631...
Starting epoch 632...
Starting epoch 633...
Starting epoch 634...
Starting epoch 635...
Starting epoch 636...
Starting epoch 637...
Starting epoch 638...
Starting epoch 639...
Starting epoch 640...
Starting epoch 641...
Starting epoch 642...
Starting epoch 643...
Starting epoch 644...
Starting epoch 645...
Starting epoch 646...
Starting epoch 647...
Starting epoch 648...
Starting epoch 649...
Starting epoch 650...
Starting epoch 651...
Starting epoch 652...
Starting epoch 653...
Starting epoch 654...
Starting e

Starting epoch 919...
Starting epoch 920...
Starting epoch 921...
Starting epoch 922...
Starting epoch 923...
Starting epoch 924...
Starting epoch 925...
Starting epoch 926...
Starting epoch 927...
Starting epoch 928...
Starting epoch 929...
Starting epoch 930...
Starting epoch 931...
Starting epoch 932...
Starting epoch 933...
Starting epoch 934...
Starting epoch 935...
Starting epoch 936...
Starting epoch 937...
Starting epoch 938...
Starting epoch 939...
Starting epoch 940...
Starting epoch 941...
Starting epoch 942...
Starting epoch 943...
Starting epoch 944...
Starting epoch 945...
Starting epoch 946...
Starting epoch 947...
Starting epoch 948...
Starting epoch 949...
Starting epoch 950...
Starting epoch 951...
Starting epoch 952...
Starting epoch 953...
Starting epoch 954...
Starting epoch 955...
Starting epoch 956...
Starting epoch 957...
Starting epoch 958...
Starting epoch 959...
Starting epoch 960...
Starting epoch 961...
Starting epoch 962...
Starting epoch 963...
Starting e

Starting epoch 1219...
Starting epoch 1220...
Starting epoch 1221...
Starting epoch 1222...
Starting epoch 1223...
Starting epoch 1224...
Starting epoch 1225...
Starting epoch 1226...
Starting epoch 1227...
Starting epoch 1228...
Starting epoch 1229...
Starting epoch 1230...
Starting epoch 1231...
Starting epoch 1232...
Starting epoch 1233...
Starting epoch 1234...
Starting epoch 1235...
Starting epoch 1236...
Starting epoch 1237...
Starting epoch 1238...
Starting epoch 1239...
Starting epoch 1240...
Starting epoch 1241...
Starting epoch 1242...
Starting epoch 1243...
Starting epoch 1244...
Starting epoch 1245...
Starting epoch 1246...
Starting epoch 1247...
Starting epoch 1248...
Starting epoch 1249...
Starting epoch 1250...
Starting epoch 1251...
Starting epoch 1252...
Starting epoch 1253...
Starting epoch 1254...
Starting epoch 1255...
Starting epoch 1256...
Starting epoch 1257...
Starting epoch 1258...
Starting epoch 1259...
Starting epoch 1260...
Starting epoch 1261...
Starting ep

...

Saving Mask...
Starting epoch 12301...
Starting epoch 12302...
Starting epoch 12303...
Starting epoch 12304...
Starting epoch 12305...
Starting epoch 12306...
Starting epoch 12307...
Starting epoch 12308...
Starting epoch 12309...
Starting epoch 12310...
Starting epoch 12311...
Starting epoch 12312...
Starting epoch 12313...
Starting epoch 12314...
Starting epoch 12315...
Starting epoch 12316...
Starting epoch 12317...
Starting epoch 12318...
Starting epoch 12319...
Starting epoch 12320...
Starting epoch 12321...
Starting epoch 12322...
Starting epoch 12323...
Starting epoch 12324...
Starting epoch 12325...
Starting epoch 12326...
Starting epoch 12327...
Starting epoch 12328...
Starting epoch 12329...
Starting epoch 12330...
Starting epoch 12331...
Starting epoch 12332...
Starting epoch 12333...
Starting epoch 12334...
Starting epoch 12335...
Starting epoch 12336...
Starting epoch 12337...
Starting epoch 12338...
Starting epoch 12339...
Starting epoch 12340...
Starting epoch 12341...
S

Saving Mask...
Starting epoch 12601...
Starting epoch 12602...
Starting epoch 12603...
Starting epoch 12604...
Starting epoch 12605...
Starting epoch 12606...
Starting epoch 12607...
Starting epoch 12608...
Starting epoch 12609...
Starting epoch 12610...
Starting epoch 12611...
Starting epoch 12612...
Starting epoch 12613...
Starting epoch 12614...
Starting epoch 12615...
Starting epoch 12616...
Starting epoch 12617...
Starting epoch 12618...
Starting epoch 12619...
Starting epoch 12620...
Starting epoch 12621...
Starting epoch 12622...
Starting epoch 12623...
Starting epoch 12624...
Starting epoch 12625...
Starting epoch 12626...
Starting epoch 12627...
Starting epoch 12628...
Starting epoch 12629...
Starting epoch 12630...
Starting epoch 12631...
Starting epoch 12632...
Starting epoch 12633...
Starting epoch 12634...
Starting epoch 12635...
Starting epoch 12636...
Starting epoch 12637...
Starting epoch 12638...
Starting epoch 12639...
Starting epoch 12640...
Starting epoch 12641...
S

Saving Mask...
Starting epoch 12901...
Starting epoch 12902...
Starting epoch 12903...
Starting epoch 12904...
Starting epoch 12905...
Starting epoch 12906...
Starting epoch 12907...
Starting epoch 12908...
Starting epoch 12909...
Starting epoch 12910...
Starting epoch 12911...
Starting epoch 12912...
Starting epoch 12913...
Starting epoch 12914...
Starting epoch 12915...
Starting epoch 12916...
Starting epoch 12917...
Starting epoch 12918...
Starting epoch 12919...
Starting epoch 12920...
Starting epoch 12921...
Starting epoch 12922...
Starting epoch 12923...
Starting epoch 12924...
Starting epoch 12925...
Starting epoch 12926...
Starting epoch 12927...
Starting epoch 12928...
Starting epoch 12929...
Starting epoch 12930...
Starting epoch 12931...
Starting epoch 12932...
Starting epoch 12933...
Starting epoch 12934...
Starting epoch 12935...
Starting epoch 12936...
Starting epoch 12937...
Starting epoch 12938...
Starting epoch 12939...
Starting epoch 12940...
Starting epoch 12941...
S

Saving Mask...
Starting epoch 13201...
Starting epoch 13202...
Starting epoch 13203...
Starting epoch 13204...
Starting epoch 13205...
Starting epoch 13206...
Starting epoch 13207...
Starting epoch 13208...
Starting epoch 13209...
Starting epoch 13210...
Starting epoch 13211...
Starting epoch 13212...
Starting epoch 13213...
Starting epoch 13214...
Starting epoch 13215...
Starting epoch 13216...
Starting epoch 13217...
Starting epoch 13218...
Starting epoch 13219...
Starting epoch 13220...
Starting epoch 13221...


KeyboardInterrupt: 

## generate mask on multiplication

In [8]:
'''Load Data'''
train_split = 0.8
test_split = 1 - train_split

data_fp = ["generate_datasets/tmp/digit-data/simple_mul.npy",
           "generate_datasets/tmp/digit-data/simple_add.npy"]
data_mul = np.load(data_fp[0], allow_pickle=True)
data_add = np.load(data_fp[1], allow_pickle=True)
print(len(data_mul))

'''Making mul dataset larger'''
mul_factor = int(len(data_add)/len(data_mul)) + 1
data_mul = np.concatenate([data_mul for i in range(mul_factor)])
np.random.shuffle(data_mul)
data_mul = data_mul[:len(data_add)]
assert len(data_mul) == len(data_add)
print(len(data_mul))

data_len = len(data_mul)
train_split_idx = int(data_len * train_split)
train_data = data_mul[:train_split_idx]
test_data = data_mul[train_split_idx:]

train_loader = torch.utils.data.DataLoader(dataset=torch.tensor(train_data), batch_size=batchsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=torch.Tensor(test_data), batch_size=batchsize, shuffle=True)

iterator_train = iter(cycle(train_loader))
iterator_test = iter(cycle(test_loader))


In [11]:
'''Initialise logits & define loss and Optimiser'''
handler = HandleAddMul(input_dims, output_dims, dir=network_cache_dir+network_name, 
                       checkpoint=checkpoint, use_optimiser=False)
handler.refreeze_weights()

logits = []
for layer in handler.network.layers[0]:
    if isinstance(layer, torch.nn.Linear):
        layer.weight.data = layer.weight.detach()
        inter = torch.full_like(layer.weight, 0.9)
        logits.append(torch.nn.Parameter(inter, requires_grad=True))

for l in logits:    
    l.requires_grad_(requires_grad=True)
        
criterion = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(logits, lr=0.01)

'''Initialise hyper-parameters'''
NUM_EPOCHS = 20000  # NB: check for number of training epochs in paper
tau = 1  # temperature parameter, NB: check for value in paper
alpha = 0.0001/128  # regularisation parameter, NB: check for value in paper
loss_trace = []
accuracy_trace = []
avg_loss_trace = []
avg_accuracy_trace = []

'''Mask Training'''
for e in range(NUM_EPOCHS):
    print(f'Starting epoch {e}...')
    
    '''Reload weights'''
    handler.network.load_save()
    handler.refreeze_weights()
    
    '''Sampling and generating masks.'''
    U1 = torch.rand(1, requires_grad=False).to(handler.network.device)
    U2 = torch.rand(1, requires_grad=False).to(handler.network.device)
    
    
    '''Gumbel Sigmoid & Straight through'''
    samples = []
    for layer in logits:
        samples.append(torch.sigmoid((layer - torch.log(torch.log(U1) / torch.log(U2))) / tau))
        
    binaries_stop = []
    for layer in samples:
        with torch.no_grad():
            binaries_stop.append((layer > 0.5).float() - layer)
        
    binaries = []
    iterator_samples = iter(samples)
          
    for idx, layer in enumerate(binaries_stop):
        binaries.append(layer + next(iterator_samples))
    terator_binary = iter(binaries)
    
    '''Inference with masked network and backpropagation.'''
    batch = next(iterator_train)
    # Load in batch data (not binaries for one-hot input)
    inp = torch.stack([torch.stack([b[0], b[1]]) for b in batch])
    otp = torch.stack([b[2] for b in batch]).to(handler.network.device)
    
    ops = torch.stack([b[3] for b in batch])
    # Convert batch data toone-hot representation
    inp, otp_ = handler.set_batched_digits(inp, otp, ops)
    inp_ = inp.to(handler.network.device)
    otp_ = otp_.to(handler.network.device)
    
    '''Pass batch data through masked net'''
    idx = 0
    for layer in handler.network.layers[0]:
        if isinstance(layer, torch.nn.Linear):
            b = layer.bias
            w = layer.weight
            m = next(terator_binary)
            inp_ = handler.network.forward_mask_layer(inp_, m, w, b)
            idx+=1
        else:
            inp_ = layer(inp_)
    otp_pred = inp_       
    assert otp_pred.is_leaf == False
    
    '''Fetch Running Accuracy'''
    with torch.no_grad():
        otp_stck = torch.stack([otp_pred[:,:10], otp_pred[:,10:]])
        otp_argmax = torch.argmax(otp_stck, dim=2)
        otp_class = otp_argmax[0]*10 + otp_argmax[1]
        diff = otp_class - otp
        cnt = len(diff[abs(diff) == 0])
        accuracy_trace.append(cnt/(len(diff)))
        
    '''Apply regularisation to CELoss'''
    all_logits = alpha*torch.cat([layer.view(-1) for layer in logits]).to(handler.network.device)
    optimiser.zero_grad()
    loss = criterion(otp_pred, otp_).to(handler.network.device) + torch.sum(all_logits).detach()
    loss.backward()
    
    for layer in handler.network.layers[0]:
        if isinstance(layer, torch.nn.Linear):
            assert layer.weight.grad == None
    optimiser.step()
    
            
    loss_trace.append(loss.item())
    avg_loss_trace.append(np.mean(loss_trace[-100:]))
    avg_accuracy_trace.append(np.mean(accuracy_trace[-10:]))
    if e % 100 == 0:        
        plt.cla()
        plt.clf()
        plt.plot(loss_trace)
        plt.plot(avg_loss_trace)
        plt.savefig('lossplot_mul.png')
        plt.cla()
        plt.clf()
        plt.plot(accuracy_trace)
        plt.plot(avg_accuracy_trace)
        plt.savefig('accuracyplot_mul.png')
        plt.cla()
        plt.clf()
        plt.close()
    if e % 100 == 0:
        print('Saving Mask...')
        torch.save(logits, 'masks/trained_logits_mul_mask_.pt') 

... FNN Network training on cuda:0 ...
Accessing : networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
Starting epoch 0...
tensor([[0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        ...,
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109],
        [0.7109, 0.7109, 0.7109,  ..., 0.7109, 0.7109, 0.7109]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)
Saving Mask...
Starting epoch 1...
Starting epoch 2...
Starting epoch 3...
Starting epoch 4...
Starting epoch 5...
Starting epoch 6...
Starting epoch 7...
Starting epoch 8...
Starting epoch 9...
Starting epoch 10...
Starting epoch 11...
Starting epoch 12...
Starting epoch 13...
Starting epoch 14...
Starting epoch 15...
Starting epoch 16...
Starting epoch 17

Saving Mask...
Starting epoch 301...
Starting epoch 302...
Starting epoch 303...
Starting epoch 304...
Starting epoch 305...
Starting epoch 306...
Starting epoch 307...
Starting epoch 308...
Starting epoch 309...
Starting epoch 310...
Starting epoch 311...
Starting epoch 312...
Starting epoch 313...
Starting epoch 314...
Starting epoch 315...
Starting epoch 316...
Starting epoch 317...
Starting epoch 318...
Starting epoch 319...
Starting epoch 320...
Starting epoch 321...
Starting epoch 322...
Starting epoch 323...
Starting epoch 324...
Starting epoch 325...
Starting epoch 326...
Starting epoch 327...
Starting epoch 328...
Starting epoch 329...
Starting epoch 330...
Starting epoch 331...
Starting epoch 332...
Starting epoch 333...
Starting epoch 334...
Starting epoch 335...
Starting epoch 336...
Starting epoch 337...
Starting epoch 338...
Starting epoch 339...
Starting epoch 340...
Starting epoch 341...
Starting epoch 342...
Starting epoch 343...
Starting epoch 344...
Starting epoch 34

Starting epoch 610...
Starting epoch 611...
Starting epoch 612...
Starting epoch 613...
Starting epoch 614...
Starting epoch 615...
Starting epoch 616...
Starting epoch 617...
Starting epoch 618...
Starting epoch 619...
Starting epoch 620...
Starting epoch 621...
Starting epoch 622...
Starting epoch 623...
Starting epoch 624...
Starting epoch 625...
Starting epoch 626...
Starting epoch 627...
Starting epoch 628...
Starting epoch 629...
Starting epoch 630...
Starting epoch 631...
Starting epoch 632...
Starting epoch 633...
Starting epoch 634...
Starting epoch 635...
Starting epoch 636...
Starting epoch 637...
Starting epoch 638...
Starting epoch 639...
Starting epoch 640...
Starting epoch 641...
Starting epoch 642...
Starting epoch 643...
Starting epoch 644...
Starting epoch 645...
Starting epoch 646...
Starting epoch 647...
Starting epoch 648...
Starting epoch 649...
Starting epoch 650...
Starting epoch 651...
Starting epoch 652...
Starting epoch 653...
Starting epoch 654...
Starting e

Starting epoch 919...
Starting epoch 920...
Starting epoch 921...
Starting epoch 922...
Starting epoch 923...
Starting epoch 924...
Starting epoch 925...
Starting epoch 926...
Starting epoch 927...
Starting epoch 928...
Starting epoch 929...
Starting epoch 930...
Starting epoch 931...
Starting epoch 932...
Starting epoch 933...
Starting epoch 934...
Starting epoch 935...
Starting epoch 936...
Starting epoch 937...
Starting epoch 938...
Starting epoch 939...
Starting epoch 940...
Starting epoch 941...
Starting epoch 942...
Starting epoch 943...
Starting epoch 944...
Starting epoch 945...
Starting epoch 946...
Starting epoch 947...
Starting epoch 948...
Starting epoch 949...
Starting epoch 950...
Starting epoch 951...
Starting epoch 952...
Starting epoch 953...
Starting epoch 954...
Starting epoch 955...
Starting epoch 956...
Starting epoch 957...
Starting epoch 958...
Starting epoch 959...
Starting epoch 960...
Starting epoch 961...
Starting epoch 962...
Starting epoch 963...
Starting e

Starting epoch 1219...
Starting epoch 1220...
Starting epoch 1221...
Starting epoch 1222...
Starting epoch 1223...
Starting epoch 1224...
Starting epoch 1225...
Starting epoch 1226...
Starting epoch 1227...
Starting epoch 1228...
Starting epoch 1229...
Starting epoch 1230...
Starting epoch 1231...
Starting epoch 1232...
Starting epoch 1233...
Starting epoch 1234...
Starting epoch 1235...
Starting epoch 1236...
Starting epoch 1237...
Starting epoch 1238...
Starting epoch 1239...
Starting epoch 1240...
Starting epoch 1241...
Starting epoch 1242...
Starting epoch 1243...
Starting epoch 1244...
Starting epoch 1245...
Starting epoch 1246...
Starting epoch 1247...
Starting epoch 1248...
Starting epoch 1249...
Starting epoch 1250...
Starting epoch 1251...
Starting epoch 1252...
Starting epoch 1253...
Starting epoch 1254...
Starting epoch 1255...
Starting epoch 1256...
Starting epoch 1257...
Starting epoch 1258...
Starting epoch 1259...
Starting epoch 1260...
Starting epoch 1261...
Starting ep

...

Saving Mask...
Starting epoch 12301...
Starting epoch 12302...
Starting epoch 12303...
Starting epoch 12304...
Starting epoch 12305...
Starting epoch 12306...
Starting epoch 12307...
Starting epoch 12308...
Starting epoch 12309...
Starting epoch 12310...
Starting epoch 12311...
Starting epoch 12312...
Starting epoch 12313...
Starting epoch 12314...
Starting epoch 12315...
Starting epoch 12316...
Starting epoch 12317...
Starting epoch 12318...
Starting epoch 12319...
Starting epoch 12320...
Starting epoch 12321...
Starting epoch 12322...
Starting epoch 12323...
Starting epoch 12324...
Starting epoch 12325...
Starting epoch 12326...
Starting epoch 12327...
Starting epoch 12328...
Starting epoch 12329...
Starting epoch 12330...
Starting epoch 12331...
Starting epoch 12332...
Starting epoch 12333...
Starting epoch 12334...
Starting epoch 12335...
Starting epoch 12336...
Starting epoch 12337...
Starting epoch 12338...
Starting epoch 12339...
Starting epoch 12340...
Starting epoch 12341...
S

Saving Mask...
Starting epoch 12601...
Starting epoch 12602...
Starting epoch 12603...
Starting epoch 12604...
Starting epoch 12605...
Starting epoch 12606...
Starting epoch 12607...
Starting epoch 12608...
Starting epoch 12609...
Starting epoch 12610...
Starting epoch 12611...
Starting epoch 12612...
Starting epoch 12613...
Starting epoch 12614...
Starting epoch 12615...
Starting epoch 12616...
Starting epoch 12617...
Starting epoch 12618...
Starting epoch 12619...
Starting epoch 12620...
Starting epoch 12621...
Starting epoch 12622...
Starting epoch 12623...
Starting epoch 12624...
Starting epoch 12625...
Starting epoch 12626...
Starting epoch 12627...
Starting epoch 12628...
Starting epoch 12629...
Starting epoch 12630...
Starting epoch 12631...
Starting epoch 12632...
Starting epoch 12633...
Starting epoch 12634...
Starting epoch 12635...
Starting epoch 12636...
Starting epoch 12637...
Starting epoch 12638...
Starting epoch 12639...
Starting epoch 12640...
Starting epoch 12641...
S

Saving Mask...
Starting epoch 12901...
Starting epoch 12902...
Starting epoch 12903...
Starting epoch 12904...
Starting epoch 12905...
Starting epoch 12906...
Starting epoch 12907...
Starting epoch 12908...
Starting epoch 12909...
Starting epoch 12910...
Starting epoch 12911...
Starting epoch 12912...
Starting epoch 12913...
Starting epoch 12914...
Starting epoch 12915...
Starting epoch 12916...
Starting epoch 12917...
Starting epoch 12918...
Starting epoch 12919...
Starting epoch 12920...
Starting epoch 12921...
Starting epoch 12922...
Starting epoch 12923...
Starting epoch 12924...
Starting epoch 12925...
Starting epoch 12926...
Starting epoch 12927...
Starting epoch 12928...
Starting epoch 12929...
Starting epoch 12930...
Starting epoch 12931...
Starting epoch 12932...
Starting epoch 12933...
Starting epoch 12934...
Starting epoch 12935...
Starting epoch 12936...
Starting epoch 12937...
Starting epoch 12938...
Starting epoch 12939...
Starting epoch 12940...
Starting epoch 12941...
S

Saving Mask...
Starting epoch 13201...
Starting epoch 13202...
Starting epoch 13203...
Starting epoch 13204...
Starting epoch 13205...
Starting epoch 13206...
Starting epoch 13207...
Starting epoch 13208...
Starting epoch 13209...
Starting epoch 13210...
Starting epoch 13211...
Starting epoch 13212...
Starting epoch 13213...
Starting epoch 13214...
Starting epoch 13215...
Starting epoch 13216...
Starting epoch 13217...
Starting epoch 13218...
Starting epoch 13219...
Starting epoch 13220...
Starting epoch 13221...


KeyboardInterrupt: 